In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pt

In [45]:
class dist :
    def __init__(self, mean, variance) :
        self.mean = mean
        self.variance = variance
    
    # http://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
    def kl(p, q) :
        (m1, v1), (m2, v2) = (p.mean, p.variance), (q.mean, q.variance)
        m1, v1, m2, v2 = float(m1), float(v1), float(m2), float(v2)
        s1, s2 = np.sqrt(v1), np.sqrt(v2)
        return np.log(s2/s1) + (v1 + (m1 - m2)**2)/(2*v2) - 0.5
    
    # https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables
    def sum(p, q) : 
        return dist(p.mean + q.mean, p.variance + q.variance)

In [185]:
class edge :
    def __init__(self, node_a, node_b, dist = None, samples = []) :
        self.dist = dist
        self.samples = []
        self.node_a = node_a
        self.node_b = node_b
        # dist is always the skill_of_node_a - skill_of_node_b
        
    def __repr__(self) : 
        return str(self.node_a) + "-" + str(self.node_b)
    
    def has_samples(self) :
        return len(self.samples) != 0
    
    def add_sample(self, sample) :
        self.samples.append(sample)

In [191]:
class graph :
    def __init__(self, V) :
        self.__matrix = []
        self.V = V
        # we will use edges only in the upper right triangle
        for i in range(V) :
            self.__matrix.append([])
            for j in range(V) :
                self.__matrix[i].append(edge(i, j))
    
    def __index(self, x, y) :
        if (x == y) :
            raise Exception("no self edges")
        
        if (x > y) : 
            return (y, x)
        
        return (x, y)
    
    def add_sample(self, x, y, sample) :
        x, y = self.__index(x, y)
        self.__matrix[y][x].add_sample(sample)
    
    def get_edge(self, x, y) :
        x, y = self.__index(x, y)
        return self.__matrix[y][x]

    def get_sampled_paths(self, x, y) :
        return self.__get_sampled_paths_helper(x, y, {x})
    
    def __get_sampled_paths_helper(self, curr, dest, visited_vertices) :
        if curr == dest :
            return [[]]
        
        total_list_of_paths = []
        for i in range(self.V) :
            if i not in visited_vertices :
                cur_edge = self.get_edge(curr, i)
                if (cur_edge.has_samples()) :
                    new_visited_vertices = visited_vertices.copy()
                    new_visited_vertices.add(i)
                    list_of_paths = self.__get_sampled_paths_helper(i, dest, new_visited_vertices)
                    for path in list_of_paths :
                        path.append(cur_edge)
                    total_list_of_paths += list_of_paths
        
        return total_list_of_paths
        
        

In [192]:
g =  graph(3)
g.add_sample(0,1,0.1)
g.add_sample(1,2,0.1)
g.add_sample(0,2,0.1)
print(str(g.get_sampled_paths(0, 2)))

[[2-1, 1-0], [2-0]]


In [147]:

a = {1}
a.add(2)
print(a)

set([1, 2])
